# EMA-Strategy Interactive Brokers execution

In [1]:
import ibapi
import pandas as pd


print(pd.__version__)
print(ibapi.__version__)


1.2.4
9.81.1


In [2]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

import pandas as pd
import threading
import time
import datetime


class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.bardata = {} #Initialize dictionary to store bar data

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)

    def tick_df(self, reqId, contract):
        ''' custom function to init DataFrame and request Tick Data '''
        self.bardata[reqId] = pd.DataFrame(columns=['time', 'price'])
        self.bardata[reqId].set_index('time', inplace=True)
        self.reqTickByTickData(reqId, contract, "Last", 0, True)
        return self.bardata[reqId]

    def tickByTickAllLast(self, reqId, tickType, time, price, size, tickAtrribLast, exchange, specialConditions):
        if tickType == 1:
            self.bardata[reqId].loc[pd.to_datetime(time, unit='s')] = price

    def futures_contract(self, symbol, secType='FUT', exchange='CME'):
        ''' custom function to create contract '''
        contract = Contract()
        contract.symbol = symbol
        contract.secType = secType
        contract.exchange = exchange
        contract.lastTradeDateOrContractMonth = "202303"
        contract.multiplier = 20
        return contract


def run_loop():
    app.run()

def submit_order(contract, direction, inv_direction, stp_price, tp_price, qty=1):
    #Create order object
    order = Order()
    order.action = direction
    order.totalQuantity = qty
    order.orderType = 'MKT'
    order.eTradeOnly = ''
    order.firmQuoteOnly = ''
    order.orderId = app.nextorderId
    app.nextorderId += 1
    order.eTradeOnly = ''
    order.firmQuoteOnly = ''
    order.transmit = False
    
    stop_order = Order()
    stop_order.action = inv_direction
    stop_order.totalQuantity = qty
    stop_order.orderType = 'STP'
    stop_order.auxPrice = stp_price
    stop_order.orderId = app.nextorderId
    app.nextorderId += 1
    stop_order.parentId = order.orderId
    stop_order.eTradeOnly = ''
    stop_order.firmQuoteOnly = ''
    stop_order.transmit = False
    
    takeprofit_order = Order()
    takeprofit_order.action = inv_direction
    takeprofit_order.totalQuantity = qty
    takeprofit_order.orderType = 'LMT'
    takeprofit_order.lmtPrice = tp_price
    takeprofit_order.orderId = app.nextorderId
    app.nextorderId += 1
    takeprofit_order.parentId = order.orderId
    takeprofit_order.eTradeOnly = ''
    takeprofit_order.firmQuoteOnly = ''
    takeprofit_order.transmit = True
  
    #submit order
    time.sleep(1)
    print('Placing {} order'.format(direction))
    app.placeOrder(order.orderId, contract, order)
    time.sleep(1)
    print('Placing STOP-LOSS order')
    app.placeOrder(stop_order.orderId, contract, stop_order)
    time.sleep(1)
    print('Placing TAKE-PROFIT order')
    app.placeOrder(takeprofit_order.orderId, contract, takeprofit_order)
    app.nextorderId += 1
    
    
    print('Full bracket order placed')
    print('Order placed at time: ' + str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    print('Sleep Time for Next Signal (300s)')
    time.sleep(300)
    print('Sleep Time Done')
    

def check_for_trade(df, contract):

    start_time = df.index[-1] - pd.Timedelta(minutes=3)
    df.index = pd.to_datetime(df.index, unit='m')
    
    data_open = df.resample('min').first()
    data_close = df.resample('min').last()
    data_high = df.resample('min').max()
    data_low = df.resample('min').min()
    
    data = df.copy()
    
    data = data.resample('min').mean()
    data['Open'] = data_open
    data['Close'] = data_close
    data['High'] = data_high
    data['Low'] = data_low

    
    trend_lag  = 3
    

    data['ema-short'] = data['Close'].shift(1).ewm(alpha=(2/(9+1))).mean()
    data['ema-short-trend'] = data['ema-short'] - data['ema-short'].shift(trend_lag)

    data['ema-mid'] = data['Close'].shift(1).ewm(alpha=(2/(50+1))).mean()
    data['ema-mid-trend'] = data['ema-mid'] - data['ema-mid'].shift(trend_lag)
    data['lower_threshold'] = data['ema-mid'] * 0.99995
    data['higher_threshold'] = data['ema-mid'] * 1.00015

    data['ema-long'] = data['Close'].shift(1).ewm(alpha=(2/(100+1))).mean()
    data['ema-long-trend'] = data['ema-long'] - data['ema-long'].shift(trend_lag)

    def quarter_round(x):
        return round(x*4)/4
    
    # Buy signal
    if (data['ema-short-trend'][-1] > 0) and (data['ema-mid-trend'][-1] > 0) and (data['ema-long-trend'][-1] > 0):
        
        # Once trend is confirmed, evaluate if the long and short moving average have the percentage difference 
        # from the middle averge.
        # For bullish: short MA should be above higher threshold, long MA below the lower threshold.
        #if (data['ema-short'][-1] > data['higher_threshold'][i]) and (data['ema-long'][-1] < data['lower_threshold'][i]):
            


        if (data['Close'][-3] < data['ema-short'][-3]) and (data['Close'][-2] > data['ema-short'][-2]):
                
            stp_p = (data['Open'][-2] + data['Close'][-2])/2
            stp_p = quarter_round(stp_p)
            
            take_p = abs(data['High'][-2] - data['Open'][-2])
            tp_p = data['Close'][-2] + take_p
            tp_p = quarter_round(tp_p)

            print('Signal: Buy')
            print('Stop Loss: ' + str(stp_p))
            print('Take Profit: ' + str(tp_p))

            submit_order(contract, direction='BUY', inv_direction='SELL', stp_price=stp_p, tp_price=tp_p)
            return True


    # Bearish rules
    elif (data['ema-short-trend'][-1] < 0) and (data['ema-mid-trend'][-1] < 0) and (data['ema-long-trend'][-1] < 0):


        # For bearish: short MA should be below lower threshold, long MA above the higher threshold
        #if (data['ema-short'][-1] < data['lower_threshold'][-1]) and (data['ema-long'][-1] > data['higher_threshold'][-1]):
            


        if (data['Close'][-3] > data['ema-short'][-3]) and (data['Close'][-2] < data['ema-short'][-2]):
                
            stp_p = (data['Close'][-2] + data['Open'][-2])/2
            stp_p = quarter_round(stp_p)
            
            take_p = abs(data['Low'][-2] - data['Open'][-2])
            tp_p = data['Close'][-2] - take_p
            tp_p = quarter_round(tp_p)
            print('Signal: Sell')
            print('Stop Loss: ' + str(stp_p))
            print('Take Profit: ' + str(tp_p))
            submit_order(contract, direction='SELL', inv_direction='BUY', stp_price=stp_p, tp_price=tp_p)
            return True


#Main
app = IBapi()
app.nextorderId = None
app.connect('127.0.0.1', 7497, 123)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop)
api_thread.start()

#Check if the API is connected via orderid
while True:
    if isinstance(app.nextorderId, int):
        print('Connected')
        break
    else:
        print('Waiting for connection')
        time.sleep(1)

#Create contract object
NQ_contract = app.futures_contract('NQ')

#-----------------------------------------------------------#

#Request tick data for google using custom function
df = app.tick_df(401, NQ_contract)

print('Requesting tick data used for order conditions calculations')

#Verify data stream
time.sleep(10)
for i in range(100):
    if len(df) > 0:
        break
    
    time.sleep(0.3)

if i == 99:
    app.disconnect()
    raise Exception ('Error with Tick data stream')

print('Streaming tick data')

#Check if there is enough data
data_length = df.index[-1] - df.index[0]
if data_length.seconds < 300:
    time.sleep(300 - data_length.seconds)
    
print('Tick data with enough size to calculate EMAs')
print('Waiting for 3-EMA signal')
#Main loop
while True:
    check_for_trade(df, NQ_contract)
    time.sleep(1)

app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


Waiting for connection
The next valid order id is:  8
Connected
Requesting tick data used for order conditions calculations
Streaming tick data
Tick data with enough size to calculate EMAs
Waiting for 3-EMA signal
Signal: Buy
Stop Loss: 11925.5
Take Profit: 11940.25
Placing BUY order
Placing STOP-LOSS order
Placing TAKE-PROFIT order
Full bracket order placed
Order placed at time: 2023-01-23 12:25:04
Sleep Time for Next Signal (300s)


ERROR 10 202 Order Canceled - reason:


Sleep Time Done
Signal: Buy
Stop Loss: 11925.75
Take Profit: 11935.5
Placing BUY order
Placing STOP-LOSS order
Placing TAKE-PROFIT order
Full bracket order placed
Order placed at time: 2023-01-23 12:38:04
Sleep Time for Next Signal (300s)


ERROR 14 202 Order Canceled - reason:


...

ERROR 42 202 Order Canceled - reason:


Sleep Time Done
Signal: Buy
Stop Loss: 11931.5
Take Profit: 11938.25
Placing BUY order
Placing STOP-LOSS order
Placing TAKE-PROFIT order
Full bracket order placed
Order placed at time: 2023-01-23 16:00:04
Sleep Time for Next Signal (300s)


ERROR 46 202 Order Canceled - reason:


Sleep Time Done


KeyboardInterrupt: 

- New strat
- Optimization
- Storage structure
- Fix when error happens rerun code